# Objective

- Implement a task-oriented collaborative workflow of agents to achieve a business objective using `crewai`

# Setup

## Installation

In [1]:
! pip install -q crewai==0.80.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.7/197.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.0/236.0 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

## Imports

In [2]:
import json
import os

from crewai import Agent, Task, Crew, Process, LLM

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [3]:
with open('config-azure.json') as f:
    configs = f.read()

In [4]:
creds = json.loads(configs)

In [18]:
default_llm = LLM(
    model='azure/gpt-4o-mini',
    base_url=creds["AZURE_OPENAI_ENDPOINT"],
    api_key=creds["AZURE_OPENAI_KEY"],
    api_version='2024-02-01',
    temperature=0
)

# Business Objective: Lead Scoring

Consider a tax software company that sells accounting and tax planning products for its enterprise customers. They have two core products - TaxEdge Pro and TaxEdge Analytics. The company wants to implement a lead scoring system for the B2B leads that show interest in the company by filling in a lead form on their website.

To score leads, the company wants to use its ideal customer profile, the products it has and match them to the customer needs as mentioned in lead form responses. The company wants to make lead followup more effective by reaching out personally only to those leads who have a high chance of conversion.

# Solution Implementation

The process of scoring form responses from leads follows a clear business process and hence is apt for a crew of agents approach. Tasks that form this business process are:
1. Extract key information from the form response.
2. Research the leads' background, industry and outlook and augment information in the form response.
3. Score the lead using the information in the form and the research conducted in step 2.
4. Prepare a plan for a company representative to approach the lead if they are deemed to qualify for a follow-up call.

For these tasks that we need to execute in sequence, we attach a specialized agent to each task and assemble a crew.

Let us now implement this plan using abstractions from `crew.ai`.

**Inputs**

In [19]:
company = 'TaxEdge Solutions'
product_names = 'TaxEdge Pro & TaxEdge Analytics'

product_descriptions = """
TaxEdge Pro is a comprehensive tax management software designed to simplify corporate tax filing and compliance processes. It offers features such as automated tax calculations, integrated payroll tax management, and seamless reporting tools to ensure businesses remain compliant with ever-changing tax laws.
TaxEdge Analytics is an advanced reporting and insights tool that helps businesses gain a deeper understanding of their tax data. It enables businesses to analyze tax trends, optimize deductions, and generate customized reports to support better financial decision-making.
"""

ideal_customer_profile_description = """
The ideal customers for TaxEdge Solutions are medium to large enterprises, including accounting firms and corporate finance departments, that manage complex tax filings and compliance requirements. These customers value efficiency, accuracy, and robust reporting tools to meet both regulatory demands and internal auditing standards. They typically face challenges such as staying up-to-date with tax law changes, managing multiple tax jurisdictions, and reducing the time spent on tax preparation and reporting.
"""

form_response = """
**Form Response**

1. **Name of your organization:** Alpha Financial Services Inc.
2. **Contact person's name and designation:** Rachel Green, Tax Compliance Manager
3. **Email address and phone number:** rachel.green@alphafinancial.com, (123) 456-7890
4. **What is the size of your company?** 500 employees
5. **How many employees are involved in tax-related processes?** 12 employees
6. **Do you currently use any tax management software? If yes, please specify.** Yes, we currently use TaxWise but are exploring alternatives for better reporting and analytics.
7. **What are your biggest challenges in tax compliance and reporting?** Keeping up with changing tax regulations across multiple jurisdictions, ensuring accuracy in tax calculations, and reducing the time spent on preparing quarterly reports.
8. **Which product are you most interested in: TaxEdge Pro, TaxEdge Analytics, or both?** Both products
9. **Would you like to schedule a demo or consultation?** Yes, I would like to schedule a demo next week.
"""

## Agents

The first step is to assemble individual agents specialized in each of the tasks identified to be executed to achieve the business objective. We define three agents below:
- Lead Analysis Agent: Analyzes form responses and extracts information needed for research and scoring
- Research Agent: Conducts research on the leads' company and industry. This research could use external tools such as search engines.
- Scoring and Planning Agent: Computes a lead score and assembles a product pitch inorder to position the correct product to the customer based on form response information and research conducted by the research agent.

Let us now define these agents using abstractions provided by `crewai`.

In [20]:
lead_analysis_agent = Agent(
    role="Lead Analysis Expert",
    goal=f"""
    You work at {company}, where you main goal is to analyze leads form responses
    to extract essential data for further research and scoring.
    """,
    backstory="""
    As a Lead Analysis Expert, you have an eye for detail and a knack for
    identifying key information. With your analytical skills, you sift through
    form responses to prepare data for in-depth research, setting the stage for
    effective lead qualification.
    """,
    llm=default_llm,
    memory=True,
    verbose=True
)

In [21]:
research_agent = Agent(
    role="Industry Research Specialist",
    goal="""
    Conduct preliminary research on the lead's industry, company size, and AI
    use case to provide a solid foundation for scoring.
    """,
    backstory=f"""
    You're an Industry Research Specialist at {company} who thrives on digging
    deep into market trends, company backgrounds, and potential use cases for AI.
    Your research uncovers vital information that informs the scoring and
    planning process, ensuring a tailored approach to each lead.
    """,
    llm=default_llm,
    memory=True,
    verbose=True
)

In [22]:
scoring_and_planning_agent = Agent(
    role="Lead Scoring and Strategic Planner",
    goal="""
    Utilize analyzed data and research findings to score leads and suggest an
    appropriate plan.
    """,
    backstory=f"""
    As the Lead Scoring and Strategic Planner at {company} you combine insights
    from lead analysis and research to score leads accurately and align them with the
    optimal offering. Your strategic vision and scoring expertise ensure that
    potential leads are matched with solutions that meet their specific needs.
    """,
    llm=default_llm,
    memory=True,
    verbose=True
)

Each of these agents has a role, a goal and a backstory that together define the system message for the LLM attached to the agent.

## Tasks

With the agents in place, we can now design the tasks these agents should execute in order. These tasks are instances of the `Task` class from `crewai` and further describe the specific task in detail adding in specifics of the company and its products. Each task also has a clear direction on the expected output from the agent executing the task.

In [23]:
lead_analysis_task = Task(
  description=f"""
   Analyze the lead's form responses to extract critical information that
   might be useful for scoring them, planning a strategy to pitch them,
   or anything that might be helpful for taking next steps on selling
   them our {product_names}.

   We at {company} are more focused on {ideal_customer_profile_description}.

   Let's not spend much time in leads that are not a good fit for us.
   Short answers from leads are a yellow flag

   Lead Form Responses:
   {form_response}

   Product Descriptions:
   {product_descriptions}
  """,
  expected_output="""
  Highlighting key implict or explicit information from the lead's form
  responses that can be used for further research and scoring.
  We are looking for insights that can help us understand the lead's
  industry, company, and potential use cases.
  """,
  agent=lead_analysis_agent
)

In [24]:
research_task = Task(
    description=f"""
    Using the information gathered from the lead's data, conduct preliminary
    research on the lead's industry, company background, and potential use
    cases for {product_names}.
    Focus on finding relevant data that can aid in scoring the lead and
    planning a strategy to pitch them {product_names}.

    Let's not spend much time in leads that are not a good fit for us.
    Short answers from leads are a yellow flag.

    Lead Form Responses:
    {form_response}

    Product Descriptions:
    {product_descriptions}
    """,
    expected_output=f"""
    A research report that includes:
    - Industry overview and trends relevant to the lead's company.
    - Company-specific information like size, market position, and recent news.
    - Potential {product_names} use cases and benefits specific to the lead's industry and company.
    - Any other insights that would aid in scoring and planning.
    The report should be concise and actionable.
    """,
    agent=research_agent,
    output_file='research-report.md'
)

In [25]:
scoring_and_planning_task = Task(
    description=f"""
    Utilize the context and the lead's form response to score the lead.
    Consider factors such as industry relevance, company size, specific {product_names}
    use case potential, and readiness for partnership.
    Take into account the wording used on the form response and how lenghty
    the response was to score the lead.
    Based on the score, suggest the most appropriate strategy for the lead.
    Integrate this information to outline how {company} and {product_names} can
    specifically benefit this lead.

    We are more focused on {ideal_customer_profile_description}

    Let's not spend much time in leads that are not a good fit for us.
    Smaller comapnies will want to pay less.
    Short answers from leads are a yellow flag.
    Let's be a little pessimistic and focus the high scores on deals that are
    more likely to close and truly great.

    Lead Form Responses:
    {form_response}


    Product Descriptions:
    {product_descriptions}
    """,
    expected_output=f"""
    A comprehensive evaluation report for each lead that includes:
    - A score from 0 to 10
    - A tailored summary of how {company} and {product_names} can assist the lead.
    - A list of talking points and ideas for an initial email and meeting, personalized for the lead.
    This output should serve as a detailed guide for engaging with the lead effectively.
    """,
    agent=scoring_and_planning_agent,
    output_file='report.md'
)

Once the final task is executed, the output needs to be presented in a markdown format and written to a report.md file.

We can now instantiate a crew with these agents by defining the set of agents and a set of tasks that need to be solved in a sequence.

In [26]:
lead_assessment_crew = Crew(
  agents=[lead_analysis_agent, research_agent, scoring_and_planning_agent],
  tasks=[lead_analysis_task, research_task, scoring_and_planning_task],
  process=Process.sequential,
  verbose=True
)

In [27]:
# Begin the task execution
response = lead_assessment_crew.kickoff()

# Agent: Lead Analysis Expert
## Task: 
   Analyze the lead's form responses to extract critical information that
   might be useful for scoring them, planning a strategy to pitch them,
   or anything that might be helpful for taking next steps on selling
   them our TaxEdge Pro & TaxEdge Analytics.

   We at TaxEdge Solutions are more focused on 
The ideal customers for TaxEdge Solutions are medium to large enterprises, including accounting firms and corporate finance departments, that manage complex tax filings and compliance requirements. These customers value efficiency, accuracy, and robust reporting tools to meet both regulatory demands and internal auditing standards. They typically face challenges such as staying up-to-date with tax law changes, managing multiple tax jurisdictions, and reducing the time spent on tax preparation and reporting.
.

   Let's not spend much time in leads that are not a good fit for us.
   Short answers from leads are a yellow flag

   Lead Form Resp

In [28]:
print(response.raw)

**Lead Score: 9/10**

**Summary of How TaxEdge Solutions Can Assist Alpha Financial Services Inc.:**  
Alpha Financial Services Inc. is a medium to large enterprise with 500 employees, making them an ideal candidate for TaxEdge Solutions. Their current challenges in tax compliance and reporting, such as keeping up with changing regulations and ensuring accuracy, align perfectly with the capabilities of TaxEdge Pro and TaxEdge Analytics. 

- **TaxEdge Pro** can automate tax calculations and integrate payroll tax management, significantly reducing the time spent on preparing quarterly reports and ensuring compliance across multiple jurisdictions. This directly addresses their need for efficiency and accuracy.
  
- **TaxEdge Analytics** will provide advanced reporting capabilities, allowing Alpha Financial Services to generate customized reports and gain deeper insights into their tax data. This will support better financial decision-making and help optimize deductions, which is crucial f

In [29]:
lead_assessment_crew.usage_metrics

UsageMetrics(total_tokens=5093, prompt_tokens=3484, cached_prompt_tokens=0, completion_tokens=1609, successful_requests=3)